In [2]:
import requests
def download_image_from_url(url, filename):
    img_data = requests.get(url).content
    with open(filename, 'wb') as handler:
        handler.write(img_data)

## Define Hyperparameters of the experiment

In [ ]:
## Define Hyperparameters of the experiment
gradio_link = "https://.....gradio.live"

# about the LVLM using AWS Bedrock
aws_access_key_id= "yours_aws_access_key_id",
aws_secret_access_key= "yours_aws_secret_access_key", 
region_name= "yours_region_name"
model_id = "anthropic.claude-3-5-sonnet-20241022-v2:0" # model_id of the LVLM

In [3]:
import cece
from cece.queries import *
from cece.refine import *
from cece.wordnet import *

import pickle

In [4]:
with open("vg_data_random.pickle", "rb") as handle:
    data = pickle.load(handle)

In [5]:
def export_text_edits(edits):
    """
    Processes a dictionary of text edits by filtering and transforming elements based on specific criteria.

    The function expects a dictionary with three keys: "transf", "additions", and "removals".
    Each key should map to a list of elements (strings or list of strings).

    The "transf" key is expected to contain a list of tuples, where each tuple represents a pair of word lists.
    The function transforms each tuple by selecting the first word from each list in the pair that does not contain a period.

    For the "additions" and "removals" keys, which map to lists of lists of strings, the function flattens these lists and includes only those elements that do not contain a period.

    Parameters:
        edits (dict): A dictionary containing three keys:
            - "transf": A list of tuples, each containing two lists of words (word pairs).
            - "additions": A list of lists, where each inner list contains words to be added.
            - "removals": A list of lists, where each inner list contains words to be removed.

    Returns:
        dict: A dictionary with the same structure as the input but filtered and transformed based on the criteria:
            - "additions": List of words to be added, filtered to exclude words containing a period.
            - "removals": List of words to be removed, filtered to exclude words containing a period.
            - "transf": A list of transformed word pairs, each selected based on the absence of a period.
    """
    transf = []
    for e1, e2 in edits["transf"]:
        ee1, ee2 = None, None
        for e in e1:
            if "." not in e:
                ee1 = e
                break
                
        for e in e2:
            if "." not in e:
                ee2 = e
                break
        transf.append([ee1, ee2])
        
    return {
        "additions": [ee for e in edits["additions"] for ee in e if "." not in ee],
        "removals": [ee for e in edits["removals"] for ee in e if "." not in ee],
        "transf": transf
    }

In [6]:
from cece.xDataset import *
from cece.xDataset import createMSQ

dataset = []
labels = []
index_to_image_id = {}
image_id_to_index = {}
for i, (k, row) in enumerate(data.items()):
    msq = []
    for o in row["objects"]:
        msq.append (connect_term_to_wordnet(o).union([o.split(".")[0]]))
    
    dataset.append(msq)
    labels.append(row["claude-3-5-sonnet"][0][0])
    index_to_image_id[i] = k
    image_id_to_index[k] = i
    
# initialize an instance of the Dataset
ds = xDataset(dataset = dataset,
              labels = labels,
              connect_to_wordnet = False)


def get_local_edits(image_id):
    source_index = image_id_to_index[image_id]
    source_image_id = index_to_image_id[source_index]
    objects_source = [dd for d in ds.dataset[source_index].concepts for dd in d if "." not in dd]

    target_index, cost = ds.explain(ds.dataset[source_index], labels[source_index])
    target_image_id = index_to_image_id[target_index]
    cost, edits = ds.find_edits(ds.dataset[source_index], ds.dataset[target_index])
    
    edits = export_text_edits(edits)
    added_objs = edits["additions"] + [e for [_, e] in edits["transf"]]
    removed_objs = edits["removals"] + [e for [e, _] in edits["transf"]]
    return objects_source, added_objs, removed_objs

## Load the Pretrained Stable Diffusion Model

In [7]:
from edits import Edits
import ast
from editor import Editor
import boto3
from chat import Chat

from PIL import Image
import matplotlib.pyplot as plt

In [8]:
editor = Editor(gradio_link)

## LVLM

In [9]:
# Define the aws runtime and model
bedrock_runtime_client = boto3.client(
    'bedrock-runtime',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key= aws_secret_access_key,
    region_name=region_name,
)



## Classifier

In [10]:
import sys

# Select between CNN classifier or LVLM classifier
# CNN classifier code
# from places365_classifier import *
# classifier = Classifier("resnet18")


with open("processed_places_categories.pickle", "rb") as f:
    processed_categories = pickle.load(f)
    
def generate_prompt(categories):
    # Join the categories into a readable list format
    categories_str = ", ".join(categories)
    return categories_str

str_categories = generate_prompt(processed_categories)

def classify(filename):
    model_id = "anthropic.claude-3-5-sonnet-20241022-v2:0"
    classification_prompt = f"""
Classify each image in their appropriate class according to the scene they depict. 
Valid classes are {str_categories} and only these, so you need to classify the images in one of these classes.
Pay attention to the semantics that define each class.
Return me only the label of the scene depicted and nothing else.
"""
    source_classes_analyze = defaultdict(list)
    for ii in range(7):
        chat = Chat(model_id, bedrock_runtime_client) # create a chat like openning a new chat in 
        chat.add_user_message_image(classification_prompt, filename) # add a user message with an image and a text prompt
        answer_source = chat.generate().strip().lower()
        source_classes_analyze = {filename: [answer_source]}
        
    source_classes_analyze_clean = defaultdict(list, {k: list(map(lambda x: x.replace('\n', ''), v)) for k, v in source_classes_analyze.items()})
    source_classes_analyze_filtered = {key: list(filter(lambda item: item in processed_categories, items)) for key, items in source_classes_analyze_clean.items()}
    for key in source_classes_analyze_filtered:
        if not source_classes_analyze_filtered[key]:  # Checks if the list is empty
            source_classes_analyze_filtered[key] = source_classes_analyze_clean[key]
            
    source_classes_analyze = defaultdict(list, {k: (lambda v: Counter(v).most_common(1)[0][0])(v) for k, v in source_classes_analyze_filtered.items()})
        
    return source_classes_analyze[filename]
            
            

classify("imgs/random/claude/11/source.jpg")

'office_cubicles'

### Run the editor with the optimal edits

In [11]:
from prompts import prompt_single_step

import os
import shutil

def create_or_replace_dir(directory_name):
    # Check if the directory already exists
    if os.path.exists(directory_name):
        # If it exists, remove it
        shutil.rmtree(directory_name)
    
    # Create the new directory
    os.makedirs(directory_name)

def edit_claude_single_step(image_id):
    
    create_or_replace_dir(f"imgs/random/claude-3-5-sonnet/claude/{image_id}")
    source_image_path = f"imgs/random/claude-3-5-sonnet/claude/{image_id}/source.jpg"
    
    url = data[image_id]["url"]
    download_image_from_url(url, source_image_path)
    steps = []
    
    # read the objects 
    objs, added_objs, removed_objs = get_local_edits(image_id)
    
    chat = Chat(model_id, bedrock_runtime_client)

    logs = ""
    excs, i = 0, 1
    orig_label = classify(source_image_path)
    logs += f"Classification: {orig_label}\n"
    new_label = orig_label
    while (new_label == orig_label):
        try:
            prompt = prompt_single_step(objs, added_objs,removed_objs)

            chat.add_user_message_image(prompt, source_image_path) # add a user message with an image and a text prompt
            step = chat.generate()
            logs += f"\n----\nOutput LVLM: {i}\n{step}\n"
            step = ast.literal_eval(step.split("\n")[0])
            logs += f"Step: {i}\n{step}\n"
            if step[0].lower() == "add":
                new_image, mask = editor.replacer(source_image_path, step[2], step[1])
                if step[1] in added_objs:
                    added_objs.remove(step[1])
                objs.append(step[1])
                steps.append(step)

            elif step[0].lower() == "remove":
                new_image, mask = editor.replacer(source_image_path, step[1], step[2])
                if step[1] in removed_objs:
                    removed_objs.remove(step[1])
                if step[1] in objs:
                    objs.remove(step[1])
                    
                steps.append(step)

            elif step[0].lower() == "replace":
                new_image, mask = editor.replacer(source_image_path, step[1], step[2])
                if step[2] in added_objs:
                    added_objs.remove(step[2])
                objs.append(step[2])

                if step[1] in removed_objs:
                    removed_objs.remove(step[1])
                if step[1] in objs:
                    objs.remove(step[1])
                    
                steps.append(step)
            else:
                print ("Unknown action!")
                return 

            source_image_path = f"imgs/random/claude-3-5-sonnet/claude/{image_id}/step_{i}.jpg"
            new_image.save(source_image_path)
            i += 1
            new_label = classify(source_image_path)
            logs += f"Classification: {new_label}\n"

        except Exception as e:
            excs += 1
            logs += f"Exception: {e}\n"
            if excs >= 5:
                break

    logs += f"\n\n----\n\n{steps}\n\n----\n\n"
    with open(f"imgs/random/claude-3-5-sonnet/claude/{image_id}/logs.txt", "w") as handle:
        handle.write(logs)
    return steps
        
    
def classified_as(image_path, cl):
    preds = classify(image_path)
    if preds == cl:
        return True
    return False 

In [ ]:
from tqdm import tqdm

for key in tqdm(data):
    edit_claude_single_step(key)

 25%|██▌       | 127/499 [3:43:00<8:19:52, 80.62s/it]  